# EE-374

# Chapter 4
# Blocks

### 4.1 The network delay
- Let's recap the core concepts up to Chapter 3, we have:
    - Created a monetary system that allows participants to transfer money between one another
    - Ensured that participants can ony spend their own money by using an *unforgeable crypto signature scheme* to authenticate transactions
    - Enabled that participants can determine *who owns what* via gossiping transactions through the network, upon verification, into a *transaction graph* & reading the *UTXO set*. Note that the latter is the only thing that determines who owns what whereas the former just stores the history of past value transfer
- Our assumption is that since every transaction is gossiped everyone will eventually arrive at the same transaction graph and reach consensus on the UTXO set
    - The order at which transactions arrive wrt some honest node doesn't matter as long as they are gossiped within some time $\Delta$ (except for the double spending problem)
    - **Definition 13** (Network delay). The network delay parameter $\Delta$ measures the maximum time it takes for a message to travel from one honest party to every other one on the network
    - Gossiping ensures that even adversarial messages make it accross the network within $\Delta$, thus can be caught

### 4.2 Double spending
- In a nutshell it is when two transactions $\text{tx}_2,\text{tx}_2^\prime$ consume the same legitimate output from previous $\text{tx}_1$ and the gossiping of $\text{tx}_2,\text{tx}_2^\prime$ occurs in different order for different honest nodes. Causing a split truth scenario:
    - some honest nodes accept $\text{tx}_2$ because it came first, these nodes removed the output from $\text{tx}_1$ from their UTXO set then updated it with the new unspent outputs from $\text{tx}_2$ and then rejected $\text{tx}_2^\prime$ because it came by second trying to spend outputs that are not there anymore BUT the reverse scenario plays out for other nodes - accepting $\text{tx}_2^\prime$ and taking its outputs as their latest UTXO set version and rejecting $\text{tx}_2$
    - This is bad because there is no consensus on the same UTXO set so we've lost agreement on *who owns what*

### 4.3 Simple ideas don't work
- We'll list some simple ideas that come natural to solve double spending and then dismantle them one by one:
    - *Idea 1*.- Reject double spends altogether $\rightarrow$ doesn't work because $\mathcal{A}$ now only as to withold $\text{tx}_2^\prime$ for a bit (see Fig.4.2)
    - *Idea 2*.- Accept the first transaction seen $\rightarrow$ makes it easy for $\mathcal{A}$ to provoke split truth, she just has to spam Charlie and David with $\text{tx}_2, \text{tx}_2^\prime$ in different order (see Fig.4.3)
    - *Idea 3*.- Reject double spends within time $u\geq\Delta$, after $u$ accept first transaction seen $\rightarrow$ still easy for $\mathcal{A}$ to break consensus, she just has to spam Charlie with $\text{tx}_2^\prime$ within $u$ and $\Delta$ and spam Dave with $\text{tx}_2^\prime$ after $u$ but within $\Delta$, the former rejects both double spends & the latter just takes $\text{tx}_2$  as it came first
- The double spending problem is not trivial so we'll spend the next many sections accumulating weapons to tackle it

<img src="images/ch043-simple-ideas.png" width="85%">

### 4.4 Ledgers
- Since honest parties receive transactions in different order we can demand them to construct & report a **Ledger** which orders transactions sequentially
    - If reading a reported Ledger by some honest party that allows the reconstruction of the transaction graph we can land to the latest UTXO set and if other honest parties agree on the reported Ledger $\Rightarrow$ we've recovered consensus on *who owns what* against double spending
- To construct a Ledger we'll need a more sophisticated system, including more elements: 
    - *(i) The Full Node*: a piece of code identically executed by all honest parties and is in charge of **peer discovery** & **gossiping** messages. Moreover, it exposes two functionalities *read* & *write* which returns the Ledger of transactions & accepts, broadcasts and gossips new transactions, respectively
    - *(ii) The Wallet*: is the intermediary element between a human user and the full node, it can invoke *read* & *write* functionalities from the full node
    
<img src="images/ch044-ledgers.png" width="60%">

- Lets go with some defs:
    - **Definition 14** *(Ledger). A ledger of an honest party $P$ reported at time $r$, denoted $L^P_r$ is a finite sequence of transactions returned wen the honest party $P$ invokes the read functionality of its honest protocol $\Pi$*
        - *note.-* that we must speak in terms of a Ledger version reported by who? (according to $P$) at some time $r$
    - **Definition 15** *(Safety (informal)). For any two honest parties, their reported Ledgers at any point in time are equal*
    - **Definition 16** *(Liveness (informal)). In an honest party writes a transaction into its Ledger, then this transaction appears in the Ledgers of all honest parties "soon"*